# Lehrter Lab Data - For NOAA RESTORE

Data cleaning of spreadsheet on Lehrter Lab Google Drive

In [ ]:
# load libraries, suppress warnings
suppressPackageStartupMessages({
#Plots
library(ggplot2)
#To make the map
library(leaflet)
#For manipulating data frames
library(dplyr)
#Dealing with Time
library(lubridate)
#For getSheetNames
library(openxlsx)
#For reading XLSX
library(readxl)
})

# Contents
- Data Entry Protocol: text instructions
- Metadata : info on names and such
- etc.

In [ ]:
filename <- "RestoreMasterData2020-2023.xlsx"

In [ ]:
sheets <- getSheetNames(filename)
sheets

## Let's just try the Cruise Data

In [ ]:
df <- read_excel(filename,sheet="Cruise Data")

In [ ]:
head(df)

## Date and Time
Note, R recognizes Time and Date, unlike when we manually exported the CSV, BUT it adds an erroneous year.

Also, when we used CSV, R knew that lat/lon and other things were numbers.  Oh well, here goes...

## Rename the columns

In [ ]:
names(df)[names(df) == 'Cruise ID'] <- 'CruiseID'
names(df)[names(df) == 'Station ID'] <- 'StationID'
names(df)[names(df) == 'Water station'] <- 'isWaterStation'
names(df)[names(df) == 'Bottom Water'] <- 'isBottomWater'
names(df)[names(df) == 'Secchi (m)'] <- 'Secchi_m'
names(df)[names(df) == 'Depth (ft)'] <- 'Depth_ft'
names(df)[names(df) == 'Depth (m)'] <- 'Depth_m'
names(df)[names(df) == 'CTD #'] <- 'CTDnum'

In [ ]:
head(df)

## Recast variables
These need to be floating point numbers
- Latitude
- Longitude
- Secchi_m
- Depth_ft
- Depth_m

This should be integer (I think?)
- CTDnum

These should be boolean (e.g., True/False)
- isWaterStation
- isBottomWater

**Note: Check those NAs!**

In [ ]:
#numeric
df$Latitude = as.numeric(df$Latitude)
df$Longitude = as.numeric(df$Longitude)
df$Secchi_m = as.numeric(df$Secchi_m)
df$Depth_m = as.numeric(df$Depth_m)
df$Depth_ft = as.numeric(df$Depth_ft)
df$Depth_m = as.numeric(df$Depth_m)

In [ ]:
#integer
df$CTDnum = as.integer(df$CTDnum)

In [ ]:
#Boolean
df <- df %>% 
  mutate(isWaterStation = recode(isWaterStation, Y = TRUE, N = FALSE))
df <- df %>% 
  mutate(isBottomWater = recode(isBottomWater, Y = TRUE, N = FALSE))
#Note, if you run this cell twice, you'll get an error.  Restart the kernal or reread the dataframe to reset.

In [ ]:
#Time
df$Newtime <- format(as.POSIXct(df$Time), format = "%H:%M:%S")
df$Newdate <- format(as.POSIXct(df$Date), format = "%m/%d/%Y")
df$timestamp <- as.POSIXct(paste(df$Newdate, df$Newtime), format="%m/%d/%Y %H:%M:%S")

In [ ]:
head(df)

## Check the data

In [ ]:
## Define bounding box
latmin = 30.10
latmax = 30.96
lonmin = -88.75
lonmax = -87.4

In [ ]:
# Mobile Bay
leaflet(df) %>%
      addTiles() %>%
      setView(lng = -87.98733, lat = 30.50355, zoom = 9) %>%
      addRectangles(
        lng1=lonmin, lat1=latmin,
        lng2=lonmax, lat2=latmax,
        fillColor = "transparent") %>%
        addCircles(lng = ~Longitude, lat = ~Latitude, weight = 5,
                radius = 500, popup = ~StationID)


## Check with some plots

In [ ]:
#General plot options
options(repr.plot.width=25, repr.plot.height=6)
plot(df$timestamp,df$Depth_m,xaxt="n",ylab="Depth (m)",xlab="")
axis(1, df$timestamp, format(df$timestamp, "%m-%d-%Y %H:%M:%S"))

## Try for a better looking plot
I tried ggplot to get the times on the axis, but ggplot doesn't do posix.  It might look better, but I can't get times.

In [ ]:
#ggplot needs 'as.Date', but it removes the time parts
df$asDate = as.Date(df$timestamp)

In [ ]:
ggplot(df,aes(asDate,Depth_m)) + 
    geom_point() +
    scale_x_date(date_breaks = "1 month") +
    theme(axis.text.x = element_text(angle = 45, hjust = 1))

## A plot showing time
Need to zoom in, then it sort of works.


In [ ]:
#Limit the date range to try to zoom in
min = as.POSIXct("2020-05-26 00:00:00")
max = as.POSIXct("2020-06-01 00:00:00")
df <- df %>% filter(Date >=min & Date <=max)

In [ ]:
#General plot options
options(repr.plot.width=25, repr.plot.height=6)
plot(df$timestamp,df$Depth_m,xaxt="n",ylab="Depth (m)",xlab="")
axis(1, df$timestamp, format(df$timestamp, "%H:%M"))